In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils import *
from read_file import *
from indicators import *
import os

## Données et comparaison des 2 méthodes

In [2]:
## Loading data
n = 200
p = 6
filename = f"./data/2KP200-TA-0.dat"

w=np.zeros(n,dtype=int) # poids des objets
v=np.zeros((n,p),dtype=int) # utilités des objets
W = readFile(filename,w,v)

# ====== On prend un sous-ensemble du problème ====
n = 50
p = 3
w = w[:n]
v = v[:n,:p]
W = w.sum()//2
print(w.shape, v.shape, W)

(50,) (50, 3) 13299


In [7]:
from time import time
params = (n,p,v,w,W)

def gap(opt, sol):
    return (opt - sol)*100 / opt

def value(x, v):
    print(x.shape)
    return v[x==1, :].sum(0)

def step(params, mode='EU', m = 20, verbose=False):
    DM = DecisionMaker(p, mode)
    DM_opt = DM.get_opt(params)
    print('Optimal: ', value(DM_opt[0], params[2]))

    # Procedure 1
    DM.nb_questions
    print('Procedure 1')
    tps = time()
    opt = RBGS(m, params, mode=mode, DM=DM, verbose=verbose)
    tps = time() - tps
    n = DM.nb_questions
    g = gap(DM_opt[1], DM.value(opt))
    res1 = (n, g, tps)
    
    DM.nb_questions = 0

    #Procedure 2
    print('Procedure 2')
    tps = time()
    opt = RBLS(params, mode=mode, DM=DM, verbose=verbose)
    tps = time() - tps
    n = DM.nb_questions
    g = gap(DM_opt[1], DM.value(opt))
    res2 = (n, g, tps)
    
    DM.nb_questions = 0
    return res1, res2

P1 = []
P2 = []
mode = 'EU'
m = 20

for it in range(2):
    print(f'======= {it = } ========')
    res1, res2 = step(params, mode, m, verbose=True)
    P1.append(res1)
    P2.append(res2)

======= it = 0 ========
(50,)
Optimal:  [19376 18691 19154]
Procedure 1
Init EU Model
it = 1 | regret = 0.26 | [20260 19926 18312] [18773 20937 16314]
it = 2 | regret = 0.20 | [20016 19505 18468] [18315 18375 19253]
it = 3 | regret = 0.05 | [19326 18843 19077] [18804 20057 18535]
it = 4 | regret = 0.03 | [19376 18691 19154] [18315 18375 19253]
it = 5 | regret = 0.01 | [19376 18691 19154] [19562 18331 19149]
it = 6 | regret = 0.01 | [19376 18691 19154] [19131 17957 19234]
it = 7 | regret = 0.00 | [19376 18691 19154] [17771 19908 18626]
it = 8 | regret = 0.00 | [19376 18691 19154] [18832 18228 19234]
Procedure 2
Init EU Model
it = 0 | voisins: 267
solution: 0.4836888331242158
[18127 17132 16950]

it = 1 | voisins: 375
solution: 0.296657289910049
[17494 17097 17451]

it = 2 | voisins: 388
solution: 0.296657289910049
[16843 16737 17970]

it = 3 | voisins: 433
solution: 0.28021809189709224
[16174 15986 18483]

it = 4 | voisins: 299
solution: 0.06875820771040088
[16003 15962 18600]

it = 5 |

In [8]:
P1

[(8, 1.9058141555214653e-14, 46.22152590751648), (11, 0.0, 73.56898212432861)]

In [ ]:
P1 = np.array(P1)
P2 = np.array(P2)
np.savetxt('P1.txt', P1)
np.savetxt('P2.txt', P2)